# **INFO5731 Assignment: 4**

**This exercise will provide a valuable learning experience in working with text data and extracting features using various topic modeling algorithms. Key concepts such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA) and BERTopic.**



**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 100


NOTE: The output should be presented well to get **full points**

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


# **Question 1 (20 Points)**

**Dataset**: 20 Newsgroups dataset

**Dataset Link**: https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

**Consider Random 2000 rows only**

Generate K=10 topics by using LDA and LSA,
then calculate coherence score and determine the optimized K value by the coherence score. Further, summarize and visualize each topics in you own words.


In [ ]:
import random
from sklearn.datasets import fetch_20newsgroups
ds = fetch_20newsgroups(subset='all',remove=('headers','footers','quotes'))
docs = ds.data
newsdocs = random.sample(docs,2000)
print("Random {0} rows are selected".format(len(newsdocs)))

Random 2000 rows are selected


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

def preprocess(text):
  text = text.lower()
  text = re.sub(r'\s+',' ',text)
  text = re.sub(r'[^a-z\s]','',text)
  tokens = [word for word in text.split() if word not in stopwords and len(word)>2]
  return tokens


processed_docs  = [preprocess(doc) for doc in newsdocs]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install gensim
!pip install numpy==1.19.2

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Using cached numpy-1.19.2.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_above=0.5,no_below=5)
corpus  = [dictionary.doc2bow(doc)for doc in processed_docs]
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [ ]:
from gensim import models
num_topics =10
lda_model = models.LdaModel(corpus=corpus,id2word=dictionary, num_topics = num_topics ,random_state=42,passes=10)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())

[(0, '0.011*"use" + 0.008*"file" + 0.008*"windows" + 0.006*"one" + 0.006*"would" + 0.006*"get" + 0.006*"window" + 0.006*"like" + 0.005*"using" + 0.005*"problem"'), (1, '0.009*"system" + 0.008*"would" + 0.007*"get" + 0.007*"know" + 0.007*"please" + 0.007*"use" + 0.006*"available" + 0.006*"thanks" + 0.006*"anyone" + 0.006*"email"'), (2, '0.009*"god" + 0.008*"one" + 0.008*"scsi" + 0.007*"would" + 0.007*"jesus" + 0.006*"dont" + 0.005*"know" + 0.005*"drive" + 0.005*"say" + 0.005*"also"'), (3, '0.008*"would" + 0.006*"dont" + 0.006*"government" + 0.006*"people" + 0.006*"team" + 0.005*"get" + 0.005*"time" + 0.005*"even" + 0.005*"one" + 0.005*"israel"'), (4, '0.010*"would" + 0.009*"get" + 0.009*"one" + 0.007*"year" + 0.006*"game" + 0.006*"conference" + 0.006*"games" + 0.005*"first" + 0.005*"like" + 0.005*"good"'), (5, '0.010*"would" + 0.009*"one" + 0.009*"space" + 0.007*"card" + 0.006*"like" + 0.006*"dont" + 0.006*"need" + 0.005*"cable" + 0.004*"make" + 0.004*"could"'), (6, '0.010*"pts" + 0.008

In [ ]:
lsa_model = models.LsiModel(corpus=corpus,id2word=dictionary,num_topics=num_topics)

AttributeError: `scipy.sparse.sparsetools` has no attribute `csc_matvecs`; furthermore, `scipy.sparse.sparsetools` is deprecated and will be removed in SciPy 2.0.0.

In [ ]:
#calculate coherence score
from gensim.models import CoherenceModel

def compute_coherence(model,texts,dictionary,corpus,coherence='c_v'):
  coherence_model = CoherenceModel(model=model,texts=texts,dictionary=dictionary,corpus=corpus,coherence=coherence)
  return coherence_model.get_coherence()

lda_coherence = compute_coherence(lda_model,processed_docs,dictionary,corpus)
lsa_coherence = compute_coherence(lsa_model,processed_docs,dictionary,corpus)

print(f"LDA score (K=10):{lda_coherence:.4f}")
print(f"LSA score (K=10):{lsa_coherence:.4f}")

In [ ]:
lda_coherence_values = []
k_values = range(1, 11)
for k in k_values:
    temp_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, random_state=42, passes=10)
    coherence = compute_coherence(temp_model, processed_docs, dictionary, corpus)
    lda_coherence_values.append(coherence)
    print(f"K={k} --> Coherence: {coherence:.4f}")

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))
plt.plot(k_values, lda_coherence_values, marker='o')
plt.title('LDA Coherence Score for different K values')
plt.xlabel('Number of Topics (K)')
plt.ylabel('Coherence Score')
plt.xticks(k_values)
plt.show()


In [ ]:
from wordcloud import WordCloud

for t in range(num_topics):
    plt.figure()
    plt.imshow(WordCloud(background_color='white').fit_words(dict(lda_model.show_topic(t, 20))))
    plt.axis("off")
    plt.title(f"LDA Topic #{t+1}")
    plt.show()


In [ ]:
from wordcloud import WordCloud

for t in range(num_topics):
    plt.figure()
    plt.imshow(WordCloud(background_color='white').fit_words(dict(lsa_model.show_topic(t, 20))))
    plt.axis("off")
    plt.title(f"LSA Topic #{t+1}")
    plt.show()


# **BERTopic**

The following question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents.

Dataset from **assignment-3** (text dataset) .

> Dont use any custom datasets.


> Dataset must have 1000+ rows, no duplicates and null values



# **Question 2 (20 Points)**



Q2) **Generate K=10 topics by using BERTopic and then find optimal K value by the coherence score. Interpret each topic and visualize with suitable style.**

In [ ]:
! pip install bertopic --quiet

In [ ]:
from bertopic import BERTopic

AttributeError: module 'numpy' has no attribute 'dtypes'

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words("english"))



def clean_text(x):
    x = str(x)
    x = x.lower()
    x = re.sub(r'#[A-Za-z0-9]*', ' ', x)  # Removing hashtags
    x = re.sub(r'https*://.*', ' ', x)  # Removing URLs
    x = re.sub(r'@[A-Za-z0-9]+', ' ', x)  # Removing mentions (@user)
    tokens = word_tokenize(x)
    x = ' '.join([w for w in tokens if w.lower() not in stop_words])  # Removing stopwords
    x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)  # Removing punctuations
    x = re.sub(r'\d+', ' ', x)  # Removing numbers
    x = re.sub(r'\n+', ' ', x)  # Removing new lines
    x = re.sub(r'\s{2,}', ' ', x)  # Removing extra spaces
    return x.strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/Antman_reviews_annotated.csv')
df.head()

,document_id,clean_text,sentiment
0,1,a huge fan first one almost big fan second one...,positive
1,2,after entri phase pas without much set next bi...,positive
2,3,well happen the mcu run ga the last mcu film l...,positive
3,4,well ill start say wasnt bad movi it wasnt gre...,positive
4,5,i enjoy watch quantumania it mostli solid fair...,positive


In [ ]:
df['clean_text']

,clean_text
0,a huge fan first one almost big fan second one...
1,after entri phase pas without much set next bi...
2,well happen the mcu run ga the last mcu film l...
3,well ill start say wasnt bad movi it wasnt gre...
4,i enjoy watch quantumania it mostli solid fair...
...,...
3470,thi film unspeak badit actual wors etern becau...
3471,a fun onei terrif time watch antman wasp quant...
3472,a mani other point far heyday peak mcu movi an...
3473,the mcu current state absolut mess up endgam w...


In [ ]:

# Apply function to DataFrame
df['clean_text'] = df['clean_text'].apply(clean_text)
df.head()

,document_id,clean_text,sentiment
0,1,huge fan first one almost big fan second one d...,positive
1,2,entri phase pas without much set next big bad ...,positive
2,3,well happen mcu run ga last mcu film lacklust ...,positive
3,4,well ill start say wasnt bad movi wasnt great ...,positive
4,5,enjoy watch quantumania mostli solid fairli en...,positive


In [ ]:
docs = df.clean_text.to_list()


In [ ]:
topic_model = BERTopic(language="english",nr_topics=10)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,1390,0_movi_charact_even_film,"[movi, charact, even, film, mcu, kang, like, a...",[even begin somehow made movi even wors uninte...
1,1,695,1_scott_antman_film_like,"[scott, antman, film, like, mcu, look, rudd, k...",[antman wasp quantumania first film phase marv...
2,2,278,2_design_wish_effect_movi,"[design, wish, effect, movi, stellar, concept,...",[visual quit stun delight creativ design shape...
3,3,278,3_marvel_movi_wast_sorri,"[marvel, movi, wast, sorri, take, written, abs...",[convinc marvel movi written chatgpt kang some...
4,4,139,4_stori_one_seat_motion,"[stori, one, seat, motion, sensori, type, let,...",[huge fan first one almost big fan second one ...
5,5,139,5_wasnt_say_babi_dune,"[wasnt, say, babi, dune, extra, unfortunatelya...",[well ill start say wasnt bad movi wasnt great...
6,6,139,6_film_entir_quantum_univers,"[film, entir, quantum, univers, watch, never, ...",[film awfulth stori ridicul waysfirstli hope m...
7,7,139,7_ai_human_creativ_mayb,"[ai, human, creativ, mayb, one, best, employ, ...",[wonder creativ peopl call creativ busi craft ...
8,8,139,8_film_badli_activ_inspir,"[film, badli, activ, inspir, bother, purpos, s...",[thi film unspeak badit actual wors etern beca...
9,9,139,9_point_display_remind_heyday,"[point, display, remind, heyday, critic, polis...",[mani point far heyday peak mcu movi anoth rem...


In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=10, n_words = 50, width = 350, height = 350)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=12, width = 700, height = 700)

In [ ]:
!pip install gensim
!pip install --upgrade h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
!pip install numpy==1.22.4

  Using cached numpy-1.22.4-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml-cu12 25.2.1 requires numpy<3.0a0,>=1.23, but you have numpy 1.22.4 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.22.4 which is incompatible.
pylibraft-cu12 25.2.0 requires numpy<3.0a0,>=1.23, but you have numpy 1.22.4 which is incompatible.
dask-cudf-cu12 25.2.2 requires numpy<3.0a0,>=1.23, but you have numpy 1.22.4 which is incompatible.
mizani 0.13.2 requires numpy>=1.23.5, but you have numpy 1.22.4 which is incompatible.
astropy 7.0.1 requires numpy>=1.23.2, but you have numpy 1.22.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.22.4

In [ ]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

def calculate_coherence_score(data, min_k=2, max_k=10):
    coherence_scores = []

    for k in range(min_k, max_k + 1):
        # Train BERTopic model with K topics
        model = BERTopic(nr_topics=k)
        topics, _ = model.fit_transform(data)

        # Extract topic words
        topic_words = [ [word for word, _ in model.get_topic(topic)] for topic in model.get_topics().keys() if topic != -1 ]

        # Prepare data for Coherence Model
        tokenized_data = [doc.split() for doc in data]
        dictionary = Dictionary(tokenized_data)

        # Compute Coherence Score
        coherence_model = CoherenceModel(topics=topic_words, dictionary=dictionary, texts=tokenized_data, coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append((k, coherence_score))

        print(f"K={k}, Coherence Score={coherence_score}")

    return coherence_scores

# Example usage with 'tweets' dataset
coherence_scores = calculate_coherence_score(docs, min_k=2, max_k=20)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
import nltk
import matplotlib.pyplot as plt
import numpy as np

nltk.download('punkt')

# Load dataset
data = pd.read_csv('Antman_reviews_annotated.csv')

# Preprocessing
docs = data['clean_text'].tolist()

# Initialize model with 10 topics

vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(nr_topics=10, vectorizer_model=vectorizer_model)
topics, probabilities = topic_model.fit_transform(docs)

# Coherence scores for optimal K
tokenized_docs = [word_tokenize(doc.lower()) for doc in docs]
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

coherence_scores = []
K_values = list(range(5, 21))

for k in K_values:
    temp_model = BERTopic(nr_topics=k, vectorizer_model=vectorizer_model)
    temp_topics, _ = temp_model.fit_transform(docs)
    topic_words = temp_model.get_topics()
    topics_words_list = [[word for word, _ in topic_words[i]] for i in range(len(topic_words)) if topic_words[i]]
    coherence_model = CoherenceModel(topics=topics_words_list, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
    coherence_scores.append(coherence_model.get_coherence())

# Identify optimal K
optimal_K = K_values[np.argmax(coherence_scores)]

# Plot coherence scores
plt.figure(figsize=(10, 5))
plt.plot(K_values, coherence_scores, marker='o')
plt.xlabel("Number of Topics (K)")
plt.ylabel("Coherence Score")
plt.title("Coherence Score vs Number of Topics")
plt.axvline(optimal_K, color='red', linestyle='--', label=f'Optimal K={optimal_K}')
plt.legend()
plt.grid(True)
plt.show()

# Print optimal K and topic summary
print(f"Optimal number of topics: {optimal_K}")
print(topic_model.get_topic_info().head(10))


AttributeError: module 'numpy' has no attribute 'dtypes'

In [ ]:
!pip  install numpy==1.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)


# **Question 3 (25 points)**


**Dataset Link**: 20 Newsgroup Dataset (Random 2000 values)

Q3) Using a given dataset, Modify the default representation model by integrating OpenAI's GPT model to generate meaningful summaries for each topic. Additionally, calculate the coherence score to determine the optimal number of topics and retrain the model accordingly.



Usefull Link: https://maartengr.github.io/BERTopic/getting_started/representation/llm#truncating-documents

In [ ]:
!pip install bertopic openai datasets nltk gensim --quiet

In [ ]:
!pip install numpy --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.


In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
from bertopic import BERTopic
from bertopic.representation import OpenAI
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
import nltk
import openai
import os

nltk.download('punkt_tab')

In [ ]:
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load dataset (random 2000 docs from 20 Newsgroups)
data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = data.data[:2000]

In [ ]:
# Preprocess docs for coherence calculation
def preprocess(texts):
    stop_words = set(stopwords.words('english'))
    texts = [[word for word in word_tokenize(doc.lower()) if word.isalpha() and word not in stop_words] for doc in texts]
    return texts
processed_docs = preprocess(docs)
id2word = Dictionary(processed_docs)

In [ ]:
# Function to calculate coherence
def coherence_score(topic_model, docs, texts, dictionary):
    topics = topic_model.get_topics()
    topics = [[word for word, _ in topic_model.get_topic(topic)] for topic in topics if topic_model.get_topic(topic)]
    coherence_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

In [ ]:
# Find optimal number of topics based on coherence
best_coherence = -1
best_topic_model = None
optimal_topics = 0

In [ ]:

for n_topics in range(10, 51, 10):
    # Train BERTopic
    vectorizer = CountVectorizer(stop_words="english")
    topic_model = BERTopic(nr_topics=n_topics, vectorizer_model=vectorizer)
    topic_model.fit(docs)

In [ ]:
# Calculate coherence
coherence = coherence_score(topic_model, docs, processed_docs, id2word)
print(f"Number of Topics: {n_topics}, Coherence Score: {coherence}")
if coherence > best_coherence:
  best_coherence = coherence
  best_topic_model = topic_model
  optimal_topics = n_topics

print(f"Optimal Number of Topics: {optimal_topics} with Coherence: {best_coherence}")

Number of Topics: 50, Coherence Score: 0.6668259739470699
Optimal Number of Topics: 50 with Coherence: 0.6668259739470699


In [ ]:
# Integrate GPT summarizer as topic representation
import openai
from bertopic.representation import OpenAI
from openai import OpenAI as OpenAIClient

client = OpenAIClient(api_key=openai.api_key)
gpt_representation = OpenAI(client=client, chat=True,verbose=True,doc_length=100,tokenizer='vectorizer')

best_topic_model.update_topics(docs, representation_model=gpt_representation)

In [ ]:
# Display generated GPT summaries for each topic
summaries = best_topic_model.get_topic_info()
for _, row in summaries.iterrows():
    topic = row['Topic']
    summary = row['Representation']
    print(f"Topic {topic}: {summary}")

Topic 0: ['Testing communications']
Topic 1: ['Data deletion concerns']
Topic 2: ['Sports commentary discussions']
Topic 3: ['Theism and fanaticism']


# **Question 4 (35 Points)**


**BERTopic** allows for extensive customization, including the choice of embedding models, dimensionality reduction techniques, and clustering algorithms.

**Dataset Link**: 20 Newsgroup Dataset (Random 2000 values)

4)

4.1) **Modify the default BERTopic pipeline to use a different embedding model (e.g., Sentence-Transformers) and a different clustering algorithm (e.g., DBSCAN instead of HDBSCAN).

4.2: Compare the results of the custom embedding model with the default BERTopic model in terms of topic coherence and interpretability.

4.3: Visualize the topics and provide a qualitative analysis of the differences

**

Usefull Link :https://www.pinecone.io/learn/bertopic/

In [ ]:
!pip install plotly


In [ ]:
# Import libraries
from bertopic import BERTopic
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import silhouette_score
import numpy as np

# Load 20 Newsgroups Dataset (Random 2000 samples)
data = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)
docs = data.data[:2000]

# Default BERTopic model for comparison
default_model = BERTopic()
default_topics, _ = default_model.fit_transform(docs)



In [ ]:

# Custom Embedding Model (SentenceTransformer) and DBSCAN clustering
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# Apply dimensionality reduction (UMAP) explicitly
from umap import UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
reduced_embeddings = umap_model.fit_transform(embeddings)

# Custom Clustering with DBSCAN
dbscan_clusterer = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
custom_model = BERTopic(embedding_model=embedding_model,
                        umap_model=umap_model,
                        hdbscan_model=dbscan_clusterer,
                        verbose=True)

custom_topics, _ = custom_model.fit_transform(docs)

# Compare number of topics
print(f"Default Model Topics: {len(set(default_topics))}")
print(f"Custom Model Topics: {len(set(custom_topics))}")

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2025-04-08 03:43:38,192 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2025-04-08 03:48:01,136 - BERTopic - Embedding - Completed ✓
2025-04-08 03:48:01,137 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-08 03:48:09,443 - BERTopic - Dimensionality - Completed ✓
2025-04-08 03:48:09,444 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-08 03:48:09,473 - BERTopic - Cluster - Completed ✓
2025-04-08 03:48:09,479 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-08 03:48:10,181 - BERTopic - Representation - Completed ✓


Default Model Topics: 38
Custom Model Topics: 6


In [ ]:
# Default BERTopic Coherence
print("Default BERTopic Model Top 10 topics:\n")
print(default_model.get_topic_info().head(10))

# Custom BERTopic Coherence
print("\nCustom BERTopic (DBSCAN + SentenceTransformer) Top 10 topics:\n")
print(custom_model.get_topic_info().head(10))

Default BERTopic Model Top 10 topics:

   Topic  Count                         Name  \
0     -1    689             -1_the_of_to_and   
1      0    190             0_the_in_he_game   
2      1     89            1_the_bike_and_to   
3      2     86               2_of_the_is_to   
4      3     86          3_the_to_clipper_of   
5      4     60        4_window_to_error_the   
6      5     57           5_ed_israel_of_the   
7      6     45  6_drive_scsi_ide_controller   
8      7     41        7_car_the_cars_saturn   
9      8     40           8_amp_for_the_with   

                                      Representation  \
0    [the, of, to, and, in, is, that, for, it, from]   
1    [the, in, he, game, to, and, that, of, is, was]   
2      [the, bike, and, to, in, was, my, of, it, on]   
3    [of, the, is, to, and, it, in, this, msg, that]   
4  [the, to, clipper, of, be, is, and, encryption...   
5  [window, to, error, the, x11r5, this, from, an...   
6  [ed, israel, of, the, in, to, israeli

In [ ]:
# Visualize Default BERTopic Model
default_model.visualize_topics().show()

# Visualize Custom BERTopic Model
custom_model.visualize_topics().show()

# Visualize topic hierarchy
default_model.visualize_hierarchy().show()
custom_model.visualize_hierarchy().show()


## Extra Question (5 Points)

**Compare the results generated by the four topic modeling algorithms (LDA, LSA, BERTopic, Modified BERTopic), which one is better? You should explain the reasons in details.**

**This question will compensate for any points deducted in this exercise. Maximum marks for the exercise is 100 points.**

In [ ]:

# Write your code here
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(docs)

# LSA
lsa_model = TruncatedSVD(n_components=10, random_state=42)
lsa_topics = lsa_model.fit_transform(X)
lsa_score = silhouette_score(lsa_topics, np.argmax(lsa_topics, axis=1))

# LDA
lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_topics = lda_model.fit_transform(X)
lda_score = silhouette_score(lda_topics, np.argmax(lda_topics, axis=1))

# Embed documents manually for BERTopic
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# BERTopic
bertopic = BERTopic(verbose=False)
topics, probs = bertopic.fit_transform(docs, embeddings)
bertopic_score = silhouette_score(embeddings, topics)

# Modified BERTopic (custom UMAP settings)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
modified_bertopic = BERTopic(umap_model=umap_model, verbose=False)
mod_topics, _ = modified_bertopic.fit_transform(docs, embeddings)
mod_bertopic_score = silhouette_score(embeddings, mod_topics)

# Comparison
results_df = pd.DataFrame({
    "Model": ["LSA", "LDA", "BERTopic", "Modified BERTopic"],
    "Silhouette Score": [lsa_score, lda_score, bertopic_score, mod_bertopic_score]
})

# Plotting
results_df.sort_values("Silhouette Score", ascending=False).plot(
    x="Model", y="Silhouette Score", kind="bar", legend=False, title="Model Comparison"
)
plt.ylabel("Silhouette Score")
plt.grid(True)
plt.show()

results_df

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.

Consider the following points in your response:

**Learning Experience:** Describe your overall learning experience in working with text data and extracting features using various topic modeling algorithms. Did you understand these algorithms and did the implementations helped in grasping the nuances of feature extraction from text data.

**Challenges Encountered:** Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:





'''